In [1]:
import copy
import itertools as it
import numpy as np
import yaml
import os

def expand_config(dict_config):
    keys, values = zip(*dict_config.items())
    permutations_dicts = [dict(zip(keys, v)) for v in it.product(*values)]
    return permutations_dicts

# SNGP NER

In [4]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=sngp do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(dpp_config['subsample_perc']))
    
    #config_list.append('training.per_device_eval_batch_size\={}'.format(dpp_config['per_device_eval_batch_size']))
    #config_list.append('training.num_train_epochs\={}'.format(dpp_config['num_train_epochs']))
    #config_list.append('training.learning_rate\={}'.format(dpp_config['learning_rate']))

    if dpp_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

In [6]:
def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models_ner.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                subsample_perc = conf['subsample_perc']
                conf['num_train_epochs'] = 80 if  subsample_perc == 0.1 else 8
                args = ' '.join(generate_func[i](conf))
                with open(f'../workdir/sngp/{task_name}/sngp_/0.1/2021-10-11/09-46-55-0/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)
                
                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                if conf['use_selective'] == 'True':
                    new_task+=f' output_dir=../workdir/run_train_models/electra-reg-sngp/{task_name}/{subsample_perc}'
                else:
                    new_task+=f' output_dir=../workdir/run_train_models/electra-raw-sngp/{task_name}/{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/ner_scripts/run_train_models_sngp_ner.sh', 'w') as rsh:
        rsh.write(full_config)

In [7]:
tasks = ['conll2003']

train_configs = {
    'calibrate': ['False'],
    'use_selective': ['False'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1],
}
    
cuda_devices = '[1,2,3]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

# Eval SNGP NER

In [2]:
import os 

def generate_eval_sngp_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(mc_config['subsample_perc']))
    
    config_list.append('training.per_device_eval_batch_size\={}'.format(mc_config['per_device_eval_batch_size']))
    
    if mc_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    
    return config_list


def generate_bash_sngp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_conll2003_for_model_series.yaml python run_conll2003_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                subsample_perc = conf['subsample_perc']
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_ner_for_model_series/electra-reg-sngp/{task_name}/{subsample_perc}'
                else:
                    base_path=f' output_dir=../workdir/run_ner_for_model_series/electra-raw-sngp/{task_name}/{subsample_perc}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'../workdir/run_train_models/electra-reg-sngp/{task_name}/{subsample_perc}/models/{task_name}'
                else:
                    model_series_dir = f'../workdir/run_train_models/electra-raw-sngp/{task_name}/{subsample_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/ner_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [3]:
tasks = ['conll2003']

eval_configs = {
    'ue': ['sngp'],
    'calibrate': ['False'],
    'use_selective': ['False'],
    'use_spectralnorm': ['False'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1],
    'per_device_eval_batch_size': [64],
    'committee_size': [10]
}
    
cuda_devices = '[1,2,3]'

generate_bash_sngp([eval_configs], cuda_devices, tasks, [generate_eval_sngp_str_config], file_name='run_eval_sngp_ner.sh')

# Generate train scripts for sngp

In [6]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=sngp do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    if dpp_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

In [7]:
import yaml

def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                output_dir = f'../workdir/run_train_models/electra-calibrate/{task_name}'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                size = 0.0 if task_name!='sst2' else 0.1
                with open(f'../workdir/sngp/{task_name}/sngp_/{size}/2021-10-11/07-01-17-0/optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)
                
                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = conf['use_spectralnorm']
                if conf['use_selective'] == 'True':
                    new_task+=f' output_dir=../workdir/run_train_models/electra-reg-sngp/{task_name}/{validation_subsample}'
                else:
                    new_task+=f' output_dir=../workdir/run_train_models/electra-raw-sngp/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/miscl_scripts/run_train_models_sngp.sh', 'w') as rsh:
        rsh.write(full_config)

In [30]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'use_selective': ['False'],
    'use_spectralnorm': ['False'],
    'validation_subsample': [0.0],
}
    
cuda_devices = '[2,3]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

# Generate eval script for sngp

In [4]:
import os 

def generate_eval_sngp_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    if mc_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    
    return config_list

def generate_bash_sngp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg-sngp/{task_name}/{validation_subsample}'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw-sngp/{task_name}/{validation_subsample}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'../workdir/run_train_models/electra-reg-sngp/{task_name}/{validation_subsample}/models/{task_name}'
                else:
                    model_series_dir = f'../workdir/run_train_models/electra-raw-sngp/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [5]:
tasks = ['mrpc', 'cola', 'sst2']

eval_configs = {
    'ue': ['sngp'],
    'calibrate': ['False'],
    'use_selective': ['False'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.0],
    'committee_size': [10]
}
    
cuda_devices = '[1,2,3]'

generate_bash_sngp([eval_configs], cuda_devices, tasks, [generate_eval_sngp_str_config], file_name='run_eval_sngp.sh')

# Generate train script for all models

In [3]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    if dpp_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

In [4]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                output_dir = f'../workdir/run_train_models/electra-calibrate/{task_name}'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['use_selective'] == 'True' else 'raw'
                with open(f'../workdir/optimal_hyperparams/{task_name}_electra_{reg}_no_sn_optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = conf['use_spectralnorm']
                if conf['use_selective'] == 'True':
                    new_task+=f' output_dir=../workdir/run_train_models_sn/electra-reg-{use_spectralnorm}/{task_name}/{validation_subsample}'
                else:
                    new_task+=f' output_dir=../workdir/run_train_models_sn/electra-raw-{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [5]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'use_selective': ['False', 'True'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.0]
}
    
cuda_devices = '[0,2,3]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_sn.sh')

# Generate train script for NER

In [11]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['validation_subsample']))
    if dpp_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

In [12]:
def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models_ner.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                output_dir = f'../workdir/run_train_models/electra-calibrate/{task_name}'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf)) + ' data.subsample_perc\=0.1 ue.calibrate\=True'
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = conf['use_spectralnorm']
                if conf['use_selective'] == 'True':
                    new_task+=f' output_dir=../workdir/run_train_models/electra-reg-{use_spectralnorm}/{task_name}/{validation_subsample}'
                else:
                    new_task+=f' output_dir=../workdir/run_train_models/electra-raw-{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../../run_train_models_ner.sh', 'w') as rsh:
        rsh.write(full_config)

In [13]:
tasks = ['conll2003']

train_configs = {
    'calibrate': ['True'],
    'use_selective': ['True', 'False'],
    'use_spectralnorm': ['True', 'False'],
    'validation_subsample': [0.1]
}
    
cuda_devices = '[0,1,2,3]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

# Generate eval script for DPP and MC-Dropout

In [9]:
import os 

def generate_eval_dpp_str_config(dpp_config):
    config_list = []
    config_list.append('ue\={}'.format(dpp_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    if dpp_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
        
    config_list.append('ue.dropout.is_reused_mask\={}'.format(dpp_config['is_reused_mask']))
    config_list.append('ue.dropout.max_frac\={}'.format(dpp_config['max_frac']))
    config_list.append('ue.dropout.committee_size\={}'.format(dpp_config['dropout_committee_size']))
    
    config_list.append('ue.committee_size\={}'.format(dpp_config['committee_size']))
    config_list.append('ue.dropout.use_ood_sampling\={}'.format(dpp_config['use_ood_sampling']))
    
    return config_list


def generate_eval_mc_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    if mc_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')

    config_list.append('ue.dropout_subs\={}'.format(mc_config['dropout_subs']))

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    config_list.append('ue.inference_prob\={}'.format(mc_config['inference_prob']))
    
    return config_list

def generate_bash_dpp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg/{task_name}/{validation_subsample}'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw/{task_name}/{validation_subsample}'
                
                use_ood_sampling = conf['use_ood_sampling']
                max_frac = conf['max_frac']
                base_path+= f'/raw_dpp/{max_frac}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-reg/{task_name}/{validation_subsample}/'
                else:
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-raw/{task_name}/{validation_subsample}/'
                model_series_dir += os.listdir(model_series_dir)[-1]
                model_series_dir += f'/{os.listdir(model_series_dir)[-1]}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [10]:
tasks = ['mrpc', 'cola', 'sst2']

dpp_configs = {
    'ue': ['mc-dpp'],
    'max_frac' : [0.3, 0.4, 0.6],
    "committee_size": [100],
    'calibrate': ['True'],
    'use_selective': ['True', 'False'],
    'use_ood_sampling': ['False'],
    'dropout_committee_size': [20],
    'validation_subsample': [0.1, 0.0],
    'is_reused_mask': ['False'],
}
    
cuda_devices = '[2]'

generate_bash_dpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_raw_dpp_models.sh')

In [7]:
def generate_bash_mc(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg/{task_name}/0.0'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw/{task_name}/0.0'
                
                dropout_subs = conf['dropout_subs']
                base_path+= f'/mc/{dropout_subs}'
                new_task+=base_path
                if conf['use_selective'] == 'True':
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-reg/{task_name}/0.0/'
                else:
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-raw/{task_name}/0.0/'
                
                model_series_dir += os.listdir(model_series_dir)[-1]
                model_series_dir += f'/{os.listdir(model_series_dir)[-1]}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../{file_name}', 'w') as rsh:
        rsh.write(full_config)
        
mc_configs = {
    'ue': ['mc'],
    'dropout_subs': ['last', 'all'],
    "committee_size": [20],
    'calibrate': ['False'],
    'inference_prob': [0.1],
    'use_selective': ['True', 'False'],
    'validation_subsample': [0.0],
}

cuda_devices = '[3]'

generate_bash_mc([mc_configs], cuda_devices, tasks, [generate_eval_mc_str_config], file_name='run_eval_mc_models.sh')

# Generate eval script for determenistic methods

In [2]:
def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\=False')

    config_list.append('ue.use_selective\={}'.format(mc_config['use_selective']))
    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=True')
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    if mc_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    
    return config_list

In [4]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                use_spectralnorm = conf['use_spectralnorm']
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series_sn/electra-reg-{use_spectralnorm}/{task_name}/0.0'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series_sn/electra-raw-{use_spectralnorm}/{task_name}/0.0'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models_sn/electra-reg-{use_spectralnorm}/{task_name}/0.0/models/{task_name}'
                else:
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models_sn/electra-raw-{use_spectralnorm}/{task_name}/0.0/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['mahalanobis', 'mc_mahalanobis'],
    'use_selective': ['False', 'True'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['True'],
}

cuda_devices = '[1]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_single_fp_models_sn.sh')

In [11]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                use_spectralnorm = '-'+conf['use_spectralnorm'] if conf['use_spectralnorm']=='True' else ''
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg{use_spectralnorm}/{task_name}/0.0'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw{use_spectralnorm}/{task_name}/0.0'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-reg{use_spectralnorm}/{task_name}/0.0/'
                else:
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-raw{use_spectralnorm}/{task_name}/0.0/'
                
                model_series_dir += os.listdir(model_series_dir)[-1]
                model_series_dir += f'/{os.listdir(model_series_dir)[-1]}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../{file_name}', 'w') as rsh:
        rsh.write(full_config)

tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['mc_mahalanobis'],
    'use_selective': ['False', 'True'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['False', 'True'],
}

cuda_devices = '[3]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_mc_fp_models.sh')

# Generate eval script for det. methods for NER

In [27]:
def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(mc_config['use_spectralnorm']))
    config_list.append('ue.use_selective\={}'.format(mc_config['use_selective']))
    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=False')
    config_list.append('ue.calibrate\=False')
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['validation_subsample']))
    
    if mc_config['use_selective'] == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    
    return config_list

In [28]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_conll2003_for_model_series.yaml python run_conll2003_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                use_spectralnorm = '-'+conf['use_spectralnorm']
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg{use_spectralnorm}/{task_name}/0.1'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw{use_spectralnorm}/{task_name}/0.1'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-reg{use_spectralnorm}/{task_name}/0.1/models/{task_name}'
                else:
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-raw{use_spectralnorm}/{task_name}/0.1/models/{task_name}'
            
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../{file_name}', 'w') as rsh:
        rsh.write(full_config)

tasks = ['conll2003']

ue_configs = {
    'ue': ['mc_mahalanobis'],
    'use_selective': ['False', 'True'],
    'validation_subsample': [0.1],
    'use_spectralnorm': ['False', 'True'],
}

cuda_devices = '[0,1,2,3]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_mc_fp_models_ner.sh')

# Generate script for det. methods (spectral norm)

In [4]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd uncertainty-estimation/src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_glue_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                use_spectralnorm = '-'+conf['use_spectralnorm'] if conf['use_spectralnorm']=='True' else ''
                if conf['use_selective'] == 'True':
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-reg{use_spectralnorm}/{task_name}/0.0'
                else:
                    base_path=f' output_dir=../workdir/run_glue_for_model_series/electra-raw{use_spectralnorm}/{task_name}/0.0'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                if conf['use_selective'] == 'True':
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-reg{use_spectralnorm}/{task_name}/0.0/'
                else:
                    model_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_models/electra-raw{use_spectralnorm}/{task_name}/0.0/'
                
                model_series_dir += os.listdir(model_series_dir)[-1]
                model_series_dir += f'/{os.listdir(model_series_dir)[-1]}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../../{file_name}', 'w') as rsh:
        rsh.write(full_config)

tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['mc_mahalanobis'],
    'use_selective': ['False', 'True'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['True'],
}

cuda_devices = '[2]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_mc_fp_models_sn.sh')

# Generate scripts for train Deep Ensemble

In [9]:
def generate_bash(cuda_devices, tasks):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_ensemble_series.yaml python ./run_train_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'
        subsample_perc = 0.63 if task_name!='sst2' else 0.1
        args = f'data.subsample_perc\={subsample_perc} ue.calibrate\=True data.validation_subsample\=0.1'
        
        with open(f'../workdir/optimal_hyperparams/{task_name}_electra_raw_no_sn_optimal_hyperparameters.yaml', 'r') as f:
            params = yaml.load(f, Loader=yaml.FullLoader)

        num_train_epochs = params['num_train_epochs']
        args += f' training.num_train_epochs\={num_train_epochs}'
        learning_rate = params['learning_rate']
        args += f' training.learning_rate\={learning_rate}'
        per_device_train_batch_size = params['per_device_train_batch_size']
        args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
        weight_decay = params['weight_decay']
        args += f' +training.weight_decay\={weight_decay}'
        
        output_dir = f'../workdir/run_train_ensemble_series/{task_name}/'
        
        new_task = copy.deepcopy(base_arg)
        new_task += f" task_configs={task_configs} args='{args}' output_dir='{output_dir}' cuda_devices={cuda_devices}"
        full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/miscl_scripts/run_train_ensemble_models.sh', 'w') as rsh:
        rsh.write(full_config)

In [10]:
tasks = ['mrpc', 'cola', 'sst2']

cuda_devices = '[1,2,3]'

generate_bash(cuda_devices, tasks)

# Generate scripts for eval Deep Ensemble

In [5]:
def generate_bash_eval_ens(cuda_devices, tasks, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_ensemble_series.yaml python run_glue_for_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'

        new_task = copy.deepcopy(base_arg)
        new_task += f' cuda_devices={cuda_devices}'
        new_task += f' config_path=../configs/{task_configs}'

        ensemble_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_ensemble_series/{task_name}/ensembles/'
        new_task+=f' ensemble_series_dir={ensemble_series_dir}'

        full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [7]:
tasks = ['mrpc', 'cola', 'sst2']

cuda_devices = '1'

generate_bash_eval_ens(cuda_devices, tasks, 'run_eval_ensemble_series.sh')

# Generate scripts for train Deep Ensemble NER

In [8]:
def generate_bash(cuda_devices, tasks):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_ensemble_series_ner.yaml python ./run_train_ensemble_series_ner.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'
        args = 'data.subsample_perc\=0.1 ue.calibrate\=True data.subsample_perc_val\=0.1 ue.use_cache\=True ue.use_selective\=False'
        
        with open(f'../workdir/optimal_hyperparams/{task_name}_electra_raw_no_sn_optimal_hyperparameters.yaml', 'r') as f:
            params = yaml.load(f, Loader=yaml.FullLoader)

        num_train_epochs = params['num_train_epochs']
        args += f' training.num_train_epochs\={num_train_epochs}'
        learning_rate = params['learning_rate']
        args += f' training.learning_rate\={learning_rate}'
        per_device_train_batch_size = params['per_device_train_batch_size']
        args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
        weight_decay = params['weight_decay']
        args += f' +training.weight_decay\={weight_decay}'
        
        output_dir = f'../workdir/run_train_ensemble_series/{task_name}/electra'
        
        new_task = copy.deepcopy(base_arg)
        new_task += f" task_configs={task_configs} args='{args}' output_dir='{output_dir}' cuda_devices={cuda_devices}"
        full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/ner_scripts/run_train_ensemble_models_ner.sh', 'w') as rsh:
        rsh.write(full_config)

In [9]:
tasks = ['conll2003']

cuda_devices = '[1,2,3]'

generate_bash(cuda_devices, tasks)

# Generate scripts for eval Deep Ensemble NER

In [7]:
def generate_bash_eval_ens(cuda_devices, tasks, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_conll2003_for_ensemble_series.yaml python run_conll2003_for_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}_selective.yaml'

        new_task = copy.deepcopy(base_arg)
        new_task += f' cuda_devices={cuda_devices}'
        new_task += f' config_path=../configs/{task_configs}'
        
        ensemble_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_ensemble_series/{task_name}/electra/ensembles/'
        new_task+=f' ensemble_series_dir={ensemble_series_dir} '

        full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/ner_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [8]:
import os

tasks = ['conll2003']

cuda_devices = '1'

generate_bash_eval_ens(cuda_devices, tasks, 'run_eval_ensemble_series_ner.sh')

In [3]:
def generate_bash_eval_ens(cuda_devices, tasks, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_conll2003_for_ensemble_series.yaml python run_conll2003_for_mahalanobis_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'

        new_task = copy.deepcopy(base_arg)
        new_task += f' cuda_devices={cuda_devices}'
        new_task += f' config_path=../configs/{task_configs}'
        
        ensemble_series_dir = f'/mnt/users/avazhentsev/uncertainty-estimation/workdir/run_train_ensemble_series/{task_name}/electra/ensembles/'
        new_task+=f' ensemble_series_dir={ensemble_series_dir} '
        
        new_task+=f' output_dir=../workdir/run_conll2003_for_mahalanobis_ensemble_series'

        full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/ner_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)
        
import os

tasks = ['conll2003']

cuda_devices = '1'

generate_bash_eval_ens(cuda_devices, tasks, 'run_eval_mahalanobis_ensemble_series_ner.sh')

# Newsgroups ALL

In [18]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'True' if dpp_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list


def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models_20newsgroups.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                output_dir = f'../workdir/run_train_models/electra-calibrate/{task_name}'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                sn = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{sn}.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                args += f' +ue.margin\={margin}'
                lamb_intra = params['lamb_intra']
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{sn}/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)
        
tasks = ['20newsgroups']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'use_spectralnorm': ['False', 'True'],
    'validation_subsample': [0.0]
}
    
cuda_devices = '[3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_newsgroups.sh')

# DPP HP

In [7]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'True' if dpp_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(dpp_config['eval_subsample']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    return config_list

def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models_20newsgroups_dpp_hp.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                output_dir = f'../workdir/run_train_models/electra-calibrate/{task_name}'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                sn = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{sn}.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                args += f' +ue.margin\={margin}'
                lamb_intra = params['lamb_intra']
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                new_task+=f' output_dir=../workdir/run_train_models_dpp_hp/electra_{reg}_{sn}/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)
        
tasks = ['20newsgroups']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False'],
    'validation_subsample': [0.0],
    'eval_subsample': [0.2]
}
    
cuda_devices = '[2,7]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_dpp_hp.sh')

In [8]:
import os 

def generate_eval_dpp_str_config(dpp_config):
    config_list = []
    config_list.append('ue\={}'.format(dpp_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(dpp_config['eval_subsample']))

    use_selective = 'True' if dpp_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
        
    config_list.append('ue.dropout.is_reused_mask\={}'.format(dpp_config['is_reused_mask']))
    config_list.append('ue.dropout.max_frac\={}'.format(dpp_config['max_frac']))
    config_list.append('ue.dropout.committee_size\={}'.format(dpp_config['dropout_committee_size']))
    
    config_list.append('ue.committee_size\={}'.format(dpp_config['committee_size']))
    config_list.append('ue.dropout.use_ood_sampling\={}'.format(dpp_config['use_ood_sampling']))
    
    return config_list


def generate_bash_dpp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_20newsgroups_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_raw_no_sn/{task_name}/{validation_subsample}'
                
                use_ood_sampling = conf['use_ood_sampling']
                dpp_type = 'ood' if use_ood_sampling=='True' else 'dpp'
                max_frac = conf['max_frac']
                dropout_committee_size = conf['dropout_committee_size']
                base_path+= f'/ddpp_{dpp_type}_{max_frac}_{dropout_committee_size}/'
                new_task+=base_path
                
                model_series_dir = f'/notebook/uncertainty-estimation/workdir/run_train_models_dpp_hp/electra_raw_no_sn/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [11]:
tasks = ['20newsgroups']

dpp_configs = {
    'ue': ['mc-dpp'],
    'max_frac' : [0.3, 0.4, 0.5, 0.6],
    "committee_size": [100],
    'calibrate': ['True'],
    'reg_type': ['raw'],
    'use_ood_sampling': ['False', 'True'],
    'dropout_committee_size': [50],
    'validation_subsample': [0.0],
    'is_reused_mask': ['True'],
    'eval_subsample': [0.2]
}
    
cuda_devices = '[2,7]'

generate_bash_dpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_dpp_models_hp_search_20ng.sh')

# Full

In [6]:
import os 

def generate_eval_dpp_str_config(dpp_config):
    config_list = []
    config_list.append('ue\={}'.format(dpp_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    use_selective = 'True' if dpp_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
        
    config_list.append('ue.dropout.is_reused_mask\={}'.format(dpp_config['is_reused_mask']))
    config_list.append('ue.dropout.max_frac\={}'.format(dpp_config['max_frac']))
    config_list.append('ue.dropout.committee_size\={}'.format(dpp_config['dropout_committee_size']))
    
    config_list.append('ue.committee_size\={}'.format(dpp_config['committee_size']))
    config_list.append('ue.dropout.use_ood_sampling\={}'.format(dpp_config['use_ood_sampling']))
    config_list.append('training.per_device_eval_batch_size\=16')
    
    return config_list


def generate_eval_mc_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    use_selective = 'True' if mc_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(mc_config['reg_type']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')

    config_list.append('ue.dropout_subs\={}'.format(mc_config['dropout_subs']))

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    config_list.append('training.per_device_eval_batch_size\=16')
    #config_list.append('ue.inference_prob\={}'.format(mc_config['inference_prob']))
    
    return config_list

def generate_bash_dpp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_20newsgroups_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                sn = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{sn}.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                args += f' +ue.margin\={margin}'
                lamb_intra = params['lamb_intra']
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
                
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{sn}/{task_name}/{validation_subsample}'
                
                use_ood_sampling = conf['use_ood_sampling']
                dpp_type = 'ood' if use_ood_sampling=='True' else 'dpp'
                max_frac = conf['max_frac']
                dropout_committee_size = conf['dropout_committee_size']
                base_path+= f'/ddpp_{dpp_type}_best/'#_{max_frac}_{dropout_committee_size}/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_{reg}_{sn}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [7]:
tasks = ['20newsgroups']

dpp_configs = {
    'ue': ['mc-dpp'],
    'max_frac' : [0.6], # best for both ddpp
    "committee_size": [100],
    'calibrate': ['True'],
    'use_spectralnorm': ['False'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'use_ood_sampling': ['False', 'True'],
    'dropout_committee_size': [50],
    'validation_subsample': [0.0],
    'is_reused_mask': ['True'],
}
    
cuda_devices = '[4,5,6]'

generate_bash_dpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_20ng_dpp_models.sh')

In [8]:
def generate_bash_mc(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_20newsgroups_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                sn = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{sn}.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                args += f' +ue.margin\={margin}'
                lamb_intra = params['lamb_intra']
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
                
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{sn}/{task_name}/{validation_subsample}'
                base_path+= f'/mc_all/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_{reg}_{sn}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)
        
tasks = ['20newsgroups']

dpp_configs = {
    'ue': ['mc'],
    "committee_size": [20],
    'calibrate': ['False'],
    'use_spectralnorm': ['False'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'validation_subsample': [0.0],
    'dropout_subs': ['all']
}   
    
cuda_devices = '[4,5,6]'

generate_bash_mc([dpp_configs], cuda_devices, tasks, [generate_eval_mc_str_config], file_name='run_eval_20ng_mc_models.sh')

In [9]:
import os

def generate_bash_det(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_20newsgroups_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                sn = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{sn}.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                args += f' +ue.margin\={margin}'
                lamb_intra = params['lamb_intra']
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
                
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                ue = conf['ue']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{sn}/{task_name}/{validation_subsample}'
                base_path+= f'/{ue}/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_{reg}_{sn}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

        
def generate_eval_ue_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    use_selective = 'True' if mc_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(mc_config['reg_type']))
    config_list.append('ue.use_spectralnorm\=False')#.format(mc_config['use_spectralnorm']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    config_list.append('training.per_device_eval_batch_size\=16')

    return config_list

tasks = ['20newsgroups']

ue_configs = {
    'ue': ['mahalanobis'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
}

cuda_devices = '[4,5,6]'

generate_bash_det([ue_configs], cuda_devices, tasks, [generate_eval_ue_str_config], file_name='run_eval_20ng_det_models.sh')

In [3]:
import os

def generate_bash_det(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_glue_for_model_series.yaml python run_20newsgroups_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                sn = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                with open(f'../../workdir/optimal_hyperparams/{task_name}_electra_{reg}_{sn}.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                
                margin = params['margin']
                args += f' +ue.margin\={margin}'
                lamb_intra = params['lamb_intra']
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
                
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                ue = conf['ue']
                base_path=f' output_dir=../workdir/run_glue_for_model_series/electra_{reg}_{sn}/{task_name}/{validation_subsample}'
                base_path+= f'/{ue}/'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_{reg}_{sn}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                full_config += '\n' + new_task if len(full_config) else new_task
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

        
def generate_eval_ue_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    use_selective = 'True' if mc_config['reg_type'] != 'raw' else 'False'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(mc_config['reg_type']))
    config_list.append('ue.use_spectralnorm\=False')#.format(mc_config['use_spectralnorm']))
    
    if use_selective == 'True':
        config_list.append('training\=electra_base')
    else:
        config_list.append('training\=electra_base')
    config_list.append('training.per_device_eval_batch_size\=16')

    return config_list

tasks = ['20newsgroups']

ue_configs = {
    'ue': ['nuq'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['False', 'True'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
}

cuda_devices = '[4,5]'

generate_bash_det([ue_configs], cuda_devices, tasks, [generate_eval_ue_str_config], file_name='run_eval_20ng_nuq_models.sh')